Este notebook contiene las consultas enseñadas durante la presentación relaccionadas con las hipótesis formuladas.

Enlace al repositorio completo: https://github.com/pintamonas4575/GESTBD-project-MAADM-UPM

In [1]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
import mariadb
from pyhive import hive
from SPARQLWrapper import SPARQLWrapper, JSON
from pyhive import hive

# **1-GRAPHDB** Descubrimiento

In [ ]:
# Configura la URL del endpoint SPARQL
sparql = SPARQLWrapper("http://localhost:8000/repositories/GESTBD")

# Establece la consulta SPARQL para el descubrimiento de BBDD estrucuradas
sparql.setQuery("""
select  ?endpoint
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Relational_Database> .
	?instance <http://upm.es/ont/endpoint> ?endpoint .
}""")
sparql.setReturnFormat(JSON)

# Ejecuta la consulta y muestra los resultados
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    for key, value in result.items():
        for key1, value1 in value.items():
            if value1 != 'literal':
                endpoint_rl=value1
endpoint_rl

'localhost,hive'

In [ ]:
# Establece la consulta SPARQL para el descubrimiento de tablas en BBDD estrucuradas
sparql.setQuery("""
    select  ?index_name ?mapping_field
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Relational_Database> .
    ?instance <http://upm.es/ont/topic> "F1".
	?instance <http://upm.es/ont/contains_table> ?object .
    ?object <http://upm.es/ont/contains_column> ?a.
    ?object <http://upm.es/ont/name> ?index_name.
    ?a <http://upm.es/ont/name> ?mapping_field.
}
""")

# Especifica el formato de retorno (JSON)
sparql.setReturnFormat(JSON)
columns_rl=[]

# Ejecuta la consulta y muestra los resultados
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    for key, value in result.items():

        for key1, value1 in value.items():
            if value1 != 'literal':
                columns_rl.append(value1) 

columns_rl


['hive_constructors',
 'name',
 'hive_drivers',
 'forename',
 'hive_drivers',
 'driverId',
 'hive_drivers',
 'number',
 'hive_drivers',
 'surname',
 'hive_races',
 'name',
 'hive_races',
 'year',
 'hive_results',
 'position']

In [ ]:
sparql.setQuery("""
select  ?endpoint 
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Elasticsearch_Database> .
	 ?instance <http://upm.es/ont/endpoint> ?endpoint .
}""")
sparql.setReturnFormat(JSON)

# Ejecuta la consulta y muestra los resultados
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    for key, value in result.items():
        for key1, value1 in value.items():
            if value1 != 'literal':
                endpoint_elastic=value1
endpoint_elastic



'http://localhost:9200'

In [ ]:
# Establece la consulta SPARQL para el descubrimiento de la base de datos Elasticsearch
sparql.setQuery("""
select  ?index_name ?mapping_field
WHERE{
    ?instance <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://upm.es/ont/Elasticsearch_Database>.
    ?instance <http://upm.es/ont/topic> "F1".
	?instance <http://upm.es/ont/contains_index> ?object .
    ?object <http://upm.es/ont/contains_mapping_field> ?a.
    ?object <http://upm.es/ont/name> ?index_name.
    ?a <http://upm.es/ont/name> ?mapping_field.
}""")
sparql.setReturnFormat(JSON)

# Ejecuta la consulta y muestra los resultados
results = sparql.query().convert()
columns_elastic=[]
for result in results["results"]["bindings"]:
    for key, value in result.items():
        for key1, value1 in value.items():
            if value1 != 'literal':
                columns_elastic.append(value1) 
columns_elastic

['my_index',
 'mapping name',
 'my_index',
 'pilot_review',
 'my_index',
 'position',
 'my_index',
 'ranking_date']

# **1º-Consulta**

Vamos a sacar el piloto con mayor número de victorias en 2023, y de ahí su ranking medio

In [ ]:
# Conexion a Hive
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [ ]:
# Consulta en Hive
cursor.execute('''
SELECT d.forename, d.surname , COUNT(*) AS victorias
FROM hive_results r
JOIN hive_races ra ON r.raceid = ra.raceid
JOIN hive_drivers d ON r.driverid = d.driverid
WHERE r.position = 1 AND ra.year = 2023
GROUP BY d.forename, d.surname
ORDER BY victorias DESC
LIMIT 1
''')
results = cursor.fetchall()
print(results[0])
name_driver= results[0][0] +' '+ results[0][1]
print(name_driver)
cursor.close()
conn.close()

('Max', 'Verstappen', 19)
Max Verstappen


In [ ]:
es = Elasticsearch("http://localhost:9200")
# Consulta en Elasticsearch para obtener la posición media del piloto en 2023
res = es.search(index='my_index', body={
    "size": 0,
    "query": {
        "bool": {
            "must": [
                {"term": {"pilot_name": {"value": name_driver}}},
                {"range": {"ranking_date": {"gte": "2023-01-01", "lte": "2023-12-31"}}}
            ]
        }
    },
    "aggs": {
        "posicion_media": {
            "avg": {
                "field": "position"
            }
        }
    }
})

# Procesa la respuesta de Elasticsearch
if res.get("aggregations"):
    posicion_media = res["aggregations"]["posicion_media"]["value"]
    print(f"Posición media de {name_driver} en el ranking en 2023: {posicion_media}")
else:
    print("No se encontró información de posición media en Elasticsearch para este piloto en 2023.")


Posición media de Max Verstappen en el ranking en 2023: 1.826086956521739


# **2º-Consulta**

In [ ]:
es = Elasticsearch("http://localhost:9200")
# Consulta en Elasticsearch para obtener la posición de 'power ranking' más alta de Lance Stroll
res = es.search(index='my_index', body={
  'query': {
    'bool': {
      'must': [
        { 'term': { 'pilot_name': 'Lance Stroll' } }
      ],
      
    }
  },
'sort': [
    { 'position': 'asc' }
  ],
  "size":1
})
position=res['hits']['hits'][0]['_source']['position']

In [106]:
position

4

In [ ]:
# Cosulta en Hive para encontrar la posición más alta del piloto en una carrera
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()
cursor.execute('''
SELECT MIN(r.position)
FROM hive_results r
JOIN hive_drivers d ON r.driverId = d.driverId
WHERE d.forename='Lance' and d.surname='Stroll' and r.position <> 0
''')
results = cursor.fetchall()
print(results[0])
cursor.close()
conn.close()

(3,)


# 3ª Consulta

In [ ]:
# Conexion a Hive
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [ ]:
# Número de carreras por conductor en 2023
cursor.execute('''
SELECT DISTINCT d.forename, d.surname, COUNT(DISTINCT r.raceid) AS num_carreras
FROM hive_drivers d
JOIN hive_results r ON d.driverid = r.driverid
JOIN hive_races rs ON r.raceid = rs.raceid
WHERE rs.year = 2023
GROUP BY d.driverid, d.forename, d.surname
''')
results = cursor.fetchall()
for driver in results:
    print(f'{driver[0]} {driver[1]} {driver[2]}')
cursor.close()
conn.close()

Alexander Albon 22
Carlos Sainz 22
Charles Leclerc 22
Daniel Ricciardo 7
Esteban Ocon 22
Fernando Alonso 22
George Russell 22
Guanyu Zhou 22
Kevin Magnussen 22
Lance Stroll 22
Lando Norris 22
Lewis Hamilton 22
Liam Lawson 5
Logan Sargeant 22
Max Verstappen 22
Nico Hülkenberg 22
Nyck de Vries 10
Oscar Piastri 22
Pierre Gasly 22
Sergio Pérez 22
Valtteri Bottas 22
Yuki Tsunoda 22


In [ ]:
es = Elasticsearch("http://localhost:9200")
# Palabras más comunes en las reviews de un piloto durante 2023
for driver in results:

    print(f"Palabras más comunes en las reviews de {driver[0]} {driver[1]} durante la temporada 2023 ({driver[2]} carreras participadas):")
    res = es.search(index='my_index', body={
        "size": 0,  # No necesitamos los documentos, solo las agregaciones
        'query': {
             "bool": {
                "should": [
                    {"wildcard": {"pilot_name": f"*{driver[0]}*"}},  # Busca si pilot_name contiene driver[0]
                    {"wildcard": {"pilot_name": f"*{driver[1]}*"}}   # Busca si pilot_name contiene driver[1]
                ],
                "minimum_should_match": 1,  # Esto garantiza que al menos una de las condiciones se cumpla
                "must": [
                    {"range": {"ranking_date": {"gte": "2023-01-01", "lte": "2023-12-31"}}}
                ]
            }
        },
        "aggs": {
            "keywords": {
                "terms": {
                    "field": "pilot_review",  
                    "size": 10  # Número de palabras comunes a devolver
                }
            }
        }
    })

    for bucket in res['aggregations']['keywords']['buckets']:
        print(f"\t{bucket['key']}, frecuencia: {bucket['doc_count']}")

    print("-"*50)

Palabras más comunes en las reviews de Alexander Albon durante la temporada 2023 (22 carreras participadas):
	albon, frecuencia: 16
	williams, frecuencia: 14
	points, frecuencia: 13
	10, frecuencia: 9
	out, frecuencia: 9
	race, frecuencia: 9
	thai, frecuencia: 9
	top, frecuencia: 9
	weekend, frecuencia: 9
	car, frecuencia: 8
--------------------------------------------------
Palabras más comunes en las reviews de Carlos Sainz durante la temporada 2023 (22 carreras participadas):
	sainz, frecuencia: 11
	race, frecuencia: 8
	spaniard, frecuencia: 8
	weekend, frecuencia: 8
	start, frecuencia: 7
	pace, frecuencia: 6
	podium, frecuencia: 6
	second, frecuencia: 6
	sixth, frecuencia: 6
	top, frecuencia: 6
--------------------------------------------------
Palabras más comunes en las reviews de Charles Leclerc durante la temporada 2023 (22 carreras participadas):
	leclerc, frecuencia: 16
	ferrari, frecuencia: 13
	race, frecuencia: 13
	lap, frecuencia: 10
	one, frecuencia: 9
	second, frecuencia

In [63]:
es.close()